Исходные данные, промежуточные данные, модели, скрипты находятся по адресу https://drive.google.com/drive/folders/1vXjG7S3HpcbVwiLmWCEvXF_4t7pA2ubE?usp=sharing

## 0. Константы 

In [1]:
N_vect=100 # Размерность вектора признаков

## 1. Импорт

In [2]:
import os
import string
import annoy
import codecs
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import gensim
from gensim.models import Word2Vec
import numpy as np
from tqdm.notebook import tqdm
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import pickle
from csv import writer
from IPython.display import display, HTML

In [3]:
morpher = MorphAnalyzer()
sw = set(get_stop_words("ru")) # Стоп-слова
exclude = set(string.punctuation) # Знаки пунктуации

## 2. Функции

In [4]:
def preprocess_txt(line, morpher=morpher, sw=sw):
    """
    Функция формирования нормальной формы слов.
        Args:
            line(string): текст, 
            morpher(MorphAnalyzer): класс MorphAnalyzer из библиотеки pymorphy2,
            sw(set): множество стоп-слов.
        Returns:
            spls(list): список слов в нормальной форме.
    """
    spls = " ".join(i.strip() for i in line.split(',')).split('(')
    spls = " ".join(" ".join(" ".join(spls).split(')')).split('.')).split('-')
    spls = " ".join(spls).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i.replace('?', '').replace('!', '') for i in spls if 
            i not in exclude and i not in sw and i != ""]
    return spls

In [5]:
def get_vector(line, model, N_vect=N_vect):
    """
    Функция формирует вектор признаков вопроса.
        Args:
            line(string): текст, 
            model(gensim.models): класс models из библиотеки gensim,
            N_vect(int): размерность вектора.
        Returns:
            vector(array): вектор признаков.
    """
    question = preprocess_txt(line)
    n_w2v = 0
    vector = np.zeros(N_vect)
    for word in question:
        if word in model.wv:
            vector += model.wv[word]
            n_w2v += 1
    if n_w2v > 0:
        vector = vector / n_w2v
    return vector

In [6]:
def get_arr(ser, model, N_vect=N_vect):
    """
    Функция формирует массив векторов признаков для текстов.
        Args:
            ser(Series): тексты, 
            model(gensim.models): класс models из библиотеки gensim,
            N_vect(int): размерность вектора.
        Returns:
            arr(array): массив векторов признаков.
    """
    arr = np.zeros([len(ser), N_vect])
    i = 0
    for line in tqdm(ser.values):
        arr[i] = get_vector(line, model, N_vect)
        i += 1
    return arr

In [7]:
def get_words_set(ser):
    """
    Функция формирует множество слов текстов.
        Args:
            ser(Series): тексты, 
        Returns:
            sentences_set(array): множество слов текстов.
    """    
    sentences_set = set()
    for line in tqdm(ser.values):
        spls = set(preprocess_txt(line))
        sentences_set.update(spls)
    return sentences_set


In [8]:
def fit_w2v_model(ser, N_vect=N_vect):
    """
    Функция обучает модель word2vec на всех словах текстов.
        Args:
            ser(Series): тексты, 
            N_vect(int): размерность вектора.
        Returns:
            model(gensim.models): класс models из библиотеки gensim.
    """
    sentences_set = get_words_set(ser)
    sentences = [[i] for i in sentences_set]
    model = Word2Vec(sentences=sentences, vector_size=N_vect, min_count=1, window=5)
    return model


## 3. Загрузка данных и сохранение предварительно обработанных данных в файлы

### 3.1 Загрузка продуктовых вопросов

In [9]:
# Загрузим датасет с продуктовыми вопросами
prod_df = pd.read_csv('./data/ProductsDataset.csv')
# Удалим неиспользуемые признаки
prod_df.drop(['category_id', 'subcategory_id', 'properties', 'image_links'], 
             inplace=True, axis=1)
print('Всего продуктовых запросов', len(prod_df)) 

Всего продуктовых запросов 35548


### 3.2 Загрузка непродуктовых вопросов для классификатора

In [10]:
# Если файл "nonprod_QA.txt" с обработанными непродуктовыми 
# вопросами и ответами уже существует, не выполняем эту ячейку

if not os.path.isfile("./data/nonprod_QA.txt"):
    # Если файла "nonprod_QA.txt" с обработанными продуктовыми нет
    # Используем текстовый датасет из ответов mail.ru.
    question = None
    written = True
    i = 0
    # Идем по всем записям, берем первую строку как вопрос, после табуляции 
    # добавляем первый ответ
    # Cохраним вопросы в файл "nonprod_QA.txt" для экономии времени
    with codecs.open("./data/nonprod_QA.txt","w", "utf-8") as fout:
        with codecs.open("./data/Answers.txt", "r", "utf-8") as fin:
            for line in tqdm(fin):
                if line.startswith("---"):
                    written = False
                    continue
                if not written and question is not None:
                    fout.write(question.replace("\t", " ").strip() + "\t" + 
                               line.replace("\t", " "))
                    i += 1
                    written = True
                    question = None
                    continue
                if not written:
                    question = line.strip()
                    continue
                # if i > 350:
                #     break
    print('Загружено непродуктовых вопросов', i)

Загружено непродуктовых вопросов 1163342

## 4. Построение классификатора вопросов

### 4.1. Обучение модели word2vec на всех вопросах, продуктовых и непродуктовых.

Формируем датафрейм продуктовых вопросов и сохраняем в файл *'prod_df.csv'* для дальнейшего использования при поиске похожих товаров по продуктовому вопросу.

Оставляем в prod_df_to_classify только запросы для классификатора

In [11]:
prod_df_to_classify = prod_df.drop(['descrirption', 'product_id'], axis=1)
prod_df_to_classify.head(2)

,title
0,Юбка детская ORBY
1,Ботильоны


Составляем nonprod_df_to_classify с непродуктовыми вопросами для классификатора.  
Ограничим количество вопросов для обучения классификатора 36000 - примерно по числу продуктовых вопросов.

In [12]:
nonprod_df_to_classify = pd.DataFrame(columns=['title'])
i = 0
with codecs.open("./data/nonprod_QA.txt","r", "utf-8") as fin:
    for line in tqdm(fin):
        # загружаем только вопросы без ответов
        nonprod_df_to_classify.loc[len(nonprod_df_to_classify)] = [line.split('\t')[0]]
        i+=1
        if i > 36000:
            break

0it [00:00, ?it/s]

In [13]:
nonprod_df_to_classify.head(2)

,title
0,вопрос о ТДВ)) давно и хорошо отдыхаем)) ЛИЧНО...
1,Как парни относятся к цветным линзам? Если у д...


Удаляем знаки препинания и делаем лемматизацию, обучаем модель word2vec на всех наших вопросах

In [14]:
# Если файл "w2v_clsfy.model" с обученной моделью уже существует,
# загружаем обученную модель из файла
if os.path.isfile("./models/w2v_clsfy.model"):
    model_clsfy = gensim.models.Word2Vec.load("./models/w2v_clsfy.model")

# Если файла "w2v_clsfy.model" с обученной моделью еще нет
else:
    # Объединяем вопросы для обучения модели word2vec
    classify_df = pd.concat([prod_df_to_classify, nonprod_df_to_classify], 
                            ignore_index=True)
    # Обучим модель word2vec на всех вопросах (Модель классификации, 
    # обученная на векторах предложений вопросов показала лучшую метрику 
    # по сравнению с моделью, обученной на множествах слов вопросов. 
    # Поэтому здесь не используется функция fit_w2v_model)
    sentences = []
    for line in tqdm(classify_df.title.values):
        spls = preprocess_txt(line, morpher, sw)
        sentences.append(spls)
    sentences = [i for i in sentences if len(i) > 0]
    model_clsfy = Word2Vec(sentences=sentences, vector_size=100, min_count=1, window=5)        
    # сохраняем обученную модель в файл
    model_clsfy.save("w2v_clsfy.model")

### 4.2. Формирование массива данных для обучения классификатора.

In [15]:
# Если файл "total_arr.npy" уже существует,
# загружаем из файла
if os.path.isfile("./data/total_arr.npy"):
    total_arr = np.load('./data/total_arr.npy')
else:
    # Массив векторов продуктовых вопросов
    prod_arr = get_arr(prod_df_to_classify.title, model_clsfy)
    # Массив векторов непродуктовых вопросов
    nonprod_arr = get_arr(nonprod_df_to_classify.title, model_clsfy)
    # Добавляем метки классов
    prod_lbl = np.ones([len(prod_df_to_classify), 1])
    nonprod_lbl = np.zeros([len(nonprod_df_to_classify), 1])
    prod_arr_lbl = np.hstack([prod_arr, prod_lbl])
    nonprod_arr_lbl = np.hstack([nonprod_arr, nonprod_lbl])
    # Общий массив векторов 
    total_arr = np.vstack([prod_arr_lbl, nonprod_arr_lbl])
    # Загрузим массив в файл для экономии времени
    file_name = './data/total_arr.npy'
    np.save(file_name, total_arr)

### 4.3. Разделение данных на обучающую и тестовую выборки.

In [16]:
ind_list = list(range(100))
y = total_arr[:,-1]
X = total_arr[:, ind_list]

In [17]:
print(X.min(), X.max())

-4.789258321126302 3.1210029125213623


Нормализацию данных можно не проводить.

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=42)

### 4.4. Создание и обучение модели классификатора, расчет метрики accuracy на тестовой выборке.

In [19]:
lr_model = LogisticRegression(max_iter=200)
lr_model.fit (X_train,y_train)

LogisticRegression(max_iter=200)

In [20]:
predictions = lr_model.predict(X_test)
# note that we can use vector operations, because we deal with numpy tensors
accuracy = (predictions == y_test).mean()
accuracy

0.9414861402282786

In [21]:
# Сохранение модели в файл
with open('./models/LR_model.pkl', 'wb') as output:
    pickle.dump(lr_model, output)

После валидации обучим модель на всех данных

In [22]:
lr_model_full = LogisticRegression(max_iter=300)
lr_model_full.fit (X,y)
# Сохранение модели в файл
with open('./models/LR_model_full.pkl', 'wb') as output:
    pickle.dump(lr_model_full, output)

Демонстрация работы классификатора

In [23]:
line = 'рубашка'
quest_label = lr_model_full.predict(get_vector(line, model_clsfy).reshape(1, -1))[0]
if quest_label == 1:
    print('Вопрос продуктовый')
else:
    print('Вопрос непродуктовый')

Вопрос продуктовый


In [24]:
predictions = lr_model_full.predict(X_test)
# note that we can use vector operations, because we deal with numpy tensors
accuracy = (predictions == y_test).mean()
accuracy

0.9433496389471232

## 5. Обработка продуктовых вопросов.

### 5.1. Обучение модели word2vec на всех словах продуктовых вопросов.

In [25]:
# Если файл "w2v_prod.model" с обученной моделью уже существует, 
# загружаем обученную модель из файла
if os.path.isfile("./models/w2v_prod.model"):
    model_prod = gensim.models.Word2Vec.load("./models/w2v_prod.model")

# Если файла "w2v_prod.model" с обученной моделью еще нет
# обучаем модель на всех словах продуктовых вопросов
else:
    
    model_prod = fit_w2v_model(prod_df.title)
    model_prod.save("./models/w2v_prod.model")

### 5.2. Построение индекса продуктовых вопросов.

In [26]:
prod_ind = annoy.AnnoyIndex(N_vect ,'angular')
# Если файл 'prod_ind.ann' с индексом уже существует,
# загружаем его из файла, а также файл словаря index_map.pkl
if os.path.isfile("./data/prod_ind.ann"):
    prod_ind.load('./data/prod_ind.ann')
    with open('./data/index_map.pkl', 'rb') as f:
        index_map = pickle.load(f)
else:

    index_map = {}
    counter = 0

    for counter in tqdm(range(len(prod_df))):
        index_map[counter] = prod_df.iloc[counter].product_id
        line = prod_df.iloc[counter].title
        vector = get_vector(line, model_prod)
        prod_ind.add_item(counter, vector)
        counter += 1

    prod_ind.build(10)  # The number of trees. More trees gives higher search precision
    prod_ind.save('./data/prod_ind.ann')
    file_name = './data/index_map.pkl'
    with open(file_name, 'wb') as output:
        pickle.dump(index_map, output)
    

### 5.3. Создание метода ответов на продуктовые вопросы

In [27]:
def find_prod_answer(question, index_map=index_map, 
                     model_prod=model_prod): 
    vector =  get_vector(question, model_prod)
    answer_index = prod_ind.get_nns_by_vector(vector, 1)
    return index_map[answer_index[0]]

Демонстрация работы метода ответов на продуктовые вопросы

In [28]:
find_prod_answer('Юбка детская ORBY')

'58e3cfe6132ca50e053f5f82'

## 6. Обработка непродуктовых вопросов.

### 6.1. Загрузка данных и формирование nonprod_df.

In [29]:
# Если файла 'nonprod_df.csv' не существует, создаем его
if not os.path.isfile('./data/nonprod_df.csv'):
    nonprod_df = pd.DataFrame(columns=['question', 'answer'])
    nonprod_df.to_csv('./data/nonprod_df.csv', index=False)
    # Построчно формируем файл 'nonprod_df.csv' из файла nonprod_QA.txt
    i=0
    with codecs.open("./data/nonprod_QA.txt","r", "utf-8") as fin:
        for line in tqdm(fin):
            try:
                # загружаем строку вопрос-ответ
                QA_line = [line.split('\t')[0], line.split('\t')[1]]
                with open('./data/nonprod_df.csv', 'a', newline='') as f_object:
                    writer_object = writer(f_object)
                    writer_object.writerow(QA_line)
                    f_object.close()
                    # Индикация прогресса
                    if i%10000 == 0:
                        print (i)
                    i+=1
            except:
                pass
    print('Загружено непродуктовых вопросов-ответов', i)

Загружено непродуктовых вопросов-ответов 1163341

In [30]:
# Загружаем nonprod_df из файла
nonprod_df = pd.read_csv('./data/nonprod_df.csv')
nonprod_df.head()

,question,answer
0,вопрос о ТДВ)) давно и хорошо отдыхаем)) ЛИЧНО...,хомячка.... \n
1,Как парни относятся к цветным линзам? Если у д...,меня вобще прикалывает эта тема :). \n
2,"Что делать, сегодня нашёл 2 миллиона рублей? .","Если это ""счастье "" действительно на вас свали..."
3,Эбу в двенашке называется Итэлма что за эбу? .,ЭБУ — электронный блок управления двигателем а...
4,академия вампиров. сколько на даный момент час...,"4. Охотники и Жертвы, Ледяной укус, Поцелуй ть..."


### 6.2. Формирование множества слов непродуктовых вопросов.

Ввиду большого объема данных сформировать множество слов непродуктовых запросов на моем компьютере в один заход не получилось. Вопросы, импортированные из таблицы nonprod_df, были разбиты на чанки по 50 тыс. В процессе обработки выяснилось, что размер чанка можно было увеличить в 4 раза.  
Формирование чанков производилось скриптом ***get_chunk.py*** 

In [31]:
# Если файлов чанков не существует, формируем их и сохраняем в файлы
# На основе данного кода был создан скрипт get_chunk.py,
# с помощью которого были созданы файлы чанков
if not os.path.isfile("./chunks/chunk_0.pkl"):
    chunk_vol = 50000
    n_chunks = len(nonprod_df)//50000 + 1
    nonprod_word_set = set()
    for i in range(n_chunks):
        begin = i*chunk_vol
        end = min((i+1)*chunk_vol, len(nonprod_df))
        # Индикация прогресса выполнения
        print(begin,end)
        chunk_ser = nonprod_df.question[begin:end]
        chunk_word_set = get_words_set(chunk_ser)
        file_name = './chunks/chunk_'+str(i)+'.pkl'
        with open(file_name, 'wb') as output:
            pickle.dump(chunk_word_set, output)

In [32]:
# Собираем множество слов непродуктовых вопросов из 24 чанков,
# ранее записанных в файлы
nonprod_set = set()
for c in range(24):
    file_name = './chunks/chunk_'+str(c)+'.pkl'
    with open(file_name, 'rb') as f:
        chunk_set = pickle.load(f)
    nonprod_set.update(chunk_set)
print('Всего слов непродуктовых вопросов',len(nonprod_set))

Всего слов непродуктовых вопросов 866369


### 6.3. Обучение модели word2vec на всех словах непродуктовых вопросов.

In [33]:
# Если файл "w2v_nonprod.model" с обученной моделью уже существует, 
# загружаем обученную модель из файла
if os.path.isfile("./models/w2v_nonprod.model"):
    model_nonprod= gensim.models.Word2Vec.load("./models/w2v_nonprod.model")

# Если файла "w2v_nonprod.model" с обученной моделью еще нет
# обучаем модель на всех словах продуктовых вопросов
else:
    model_nonprod = fit_w2v_model(nonprod_df.question)
    # Сохраним модель 
    # (3 файла: w2v_nonprod.model, w2v_nonprod.model.syn1neg.npy, 
    # w2v_nonprod.model.wv.vectors.npy )
    model_nonprod.save("./models/w2v_nonprod.model")

### 6.4. Формирование массива векторов непродуктовых вопросов.

Ввиду большого объема данных сформировать индекс непродуктовых запросов на моем компьютере в один заход не получилось. Вопросы, импортированные из таблицы nonprod_df, были разбиты на чанки по 100 тыс, из которых были сформированы массивы векторов признаков и сохранены в файлы.  
Формирование массивов векторов производилось скриптом ***nonprod_arr.py***

In [34]:
# Собираем массив векторов признаков непродуктовых вопросов из файлов
n_chunk = 0
nonprod_arr = np.load('./arrays/nonprod_arr_'+str(n_chunk)+'.npy')
for n_chunk in range(1, 12):
    file_name = './arrays/nonprod_arr_'+str(n_chunk)+'.npy'
    curr_arr = np.load('./arrays/nonprod_arr_'+str(n_chunk)+'.npy')
    nonprod_arr = np.vstack([nonprod_arr, curr_arr])

### 6.5. Построение индекса непродуктовых вопросов.

In [35]:
nonprod_ind = annoy.AnnoyIndex(N_vect ,'angular')
# Если файл 'nonprod_ind.ann' с индексом уже существует,
# загружаем его из файла, а также файл словаря answer_map
if os.path.isfile("./data/nonprod_ind.ann"):
    nonprod_ind.load('./data/nonprod_ind.ann')
    with open('./data/answer_map.pkl', 'rb') as f:
        answer_map = pickle.load(f)
    
# Если файла 'nonprod_ind.ann' с индексом не существует, 
# собираем индекс из массива nonprod_arr
else:
    counter = 0
    answer_map = {} # Словарь ответов
    for counter in tqdm(range(nonprod_arr.shape[0])):
        vector = nonprod_arr[counter]
        nonprod_ind.add_item(counter, vector)

        answer_map[counter] = nonprod_df.iloc[counter].answer
        counter += 1

    nonprod_ind.build(10)  # The number of trees. More trees gives higher search precision
    nonprod_ind.save('./data/nonprod_ind.ann')
    file_name = './data/answer_map.pkl'
    with open(file_name, 'wb') as output:
        pickle.dump(index_map, output)    

  0%|          | 0/1163341 [00:00<?, ?it/s]

### 6.6. Создание метода ответов на непродуктовые вопросы

In [36]:
def find_nonprod_answer(question, answer_map=answer_map, 
                        model_nonprod=model_nonprod): 
    vector =  get_vector(question, model_nonprod)
    answer_index = nonprod_ind.get_nns_by_vector(vector, 1)
    return answer_map[answer_index[0]]

## 7. Создание общего метода ответа на вопрос.

In [43]:
with open('./models/LR_model_full.pkl', 'rb') as f:
    lr_model_full = pickle.load(f)

def get_answer(question, 
               lr_model_full=lr_model_full, 
               model_clsfy=model_clsfy,
               model_prod=model_prod, 
               model_nonprod=model_nonprod,
               index_map=index_map, 
               answer_map=answer_map):
    quest_label = lr_model_full.predict(get_vector(question, model_clsfy).reshape(1, -1))[0]
    if quest_label == 1:  # Вопрос продуктовый
        answer = find_prod_answer(question)
    else:                 # Вопрос непродуктовый
        answer = find_nonprod_answer(question)
    return answer

## 8. Примеры работы метода get_answer.

In [44]:
line = 'Юбка детская ORBY'
display(HTML(get_answer(line)))

In [45]:
line = 'чему равна площадь треугольника'
display(HTML(get_answer(line)))

In [46]:
line = 'С добрым утром'
display(HTML(get_answer(line)))

In [47]:
line = 'Хочу выпить'
display(HTML(get_answer(line)))

In [48]:
line = 'Хочу поесть'
display(HTML(get_answer(line)))

In [50]:
line = 'Хочу пожрать'
display(HTML(get_answer(line)))